# Value-at-Risk

## Import Libraries

In [1]:

import numpy as np
import pandas as pd
import requests
from dateutil import relativedelta
from scipy.stats import t
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Risk Free Rate 
RF = 0.05
CONF_LEVEL = 95
TIME_HORIZEN = 1


## Historical Value-at-Risk

In [2]:
def get_alphavantagekey(path):
    with open(path) as f:
        key  = f.read().strip()
    return key
KEY=get_alphavantagekey('keys/alphavantage.txt')

In [177]:
ra = 0.08
va = 0.16
rd = (1+ra)**(1/252)-1  # daily
vd = va / np.sqrt(252)  # daily

# df => higher values => normal
returns = t.rvs(loc=rd,scale=vd,size=100000,df=5)

# validate
mu = (1+np.mean(returns))**252 -1
sigma = np.std(returns) * np.sqrt(252)


In [3]:
class FinancialInstrument:
    def __init__(self, name, identifier, prices = None, returns=None,total_return=None, mu=None, sigma=None, histVaR=None, histCVaR=None,conf_level=None,time_horizen=None):
        self.name = name
        self.identifier = identifier
        self.conf_level = conf_level
        self.time_horizen = time_horizen

    def __str__(self):
        txt = f"{self.name} ({self.identifier})\n"
        txt += "="*100 + '\n'
        # txt += f"Total Return: {self.total_return*100:.2f}% | Annualized: {self.annualized_return*100:.2f}% | Volatility: {self.annualized_volatility*100:.2f}%\n"
        return txt

    def calc_HistVaR(self):
        """Historical Value-at-Risk for a confidence interval"""
        VaR = np.percentile(self.returns,100-self.conf_level) * np.sqrt(self.time_horizen)
        return VaR

    def calc_HistCVaR(self):
        """
        Historical Conditional Value-at-Risk for a confidence interval
        Find the average of the worst returns below the VaR
        """
        below_var = self.returns <= self.histVaR
        CVaR = self.returns[below_var].mean() * np.sqrt(self.time_horizen)
        return CVaR

    def plot_prices(self):
        df = self.prices
        hover_temp = "Date:%{x}"+ " Closing Price:%{y:,.0f}"
        title = f"<b>{self.name} ({self.identifier})</b><br>Return: {self.mu*100:.2f}% | Volatility: {self.sigma*100:.2f}% | TR:{self.total_return*100:.2f}%"
        fig = go.Figure(
            go.Scatter(
                x = df.index,
                y = df.close,
                line = dict(color = 'rgb(83,128,141)',width=2),
                fill='tozeroy',
                fillcolor = 'rgba(83,128,141,0.5)',
                hovertemplate = hover_temp
            )
        )
        fig.update_layout(
            template='plotly_white',
            title = title,
            yaxis_title='Closing Price' ,
            width=600, height=500
        )
        fig.show()

    def plot_histograms(self):
        """"Plot the histogram of daily returns with VaR"""

        title = f"<b>{self.name} Historical VaR ({self.time_horizen} days) </b><br>VaR:{self.histVaR*100:.2f}% | CVaR: {self.histCVaR*100:.2f}%"
        returns = self.returns *100
        min_r = min(self.returns)*100
        max_r = max(self.returns)*100

        # histogram
        fig = go.Figure(
            go.Histogram(
                x = returns,
                marker_color = 'rgba(83,128,141,0.8)'
            )
        )
        # CVaR Area
        fig.add_shape(
            type='rect',
            x0=min_r,
            x1= self.histVaR*100,
            y0=0,
            y1=1,
            fillcolor='red',
            opacity=0.1
        )
        # Non-CVaR Area
        fig.add_shape(
            type='rect',
            x1=max_r,
            x0= self.histVaR*100,
            y0=0,
            y1=1,
            fillcolor='skyblue',
            opacity=0.1
        )
        # VaR threshold
        fig.add_shape(
            type='line',
            x1=self.histVaR *100,
            x0= self.histVaR*100,
            y0=0,
            y1=1,
            line=dict(color='rgba(255,51,51,.5)'),
        )

        # VaR Threshold
        fig.add_annotation(
            x=self.histVaR*100,
            y=0.8,
            xref='x',
            yref='paper',
            text = "VaR<br>Threshold",
            ax=100,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            bordercolor = 'rgba(255,51,51,.2)',
            bgcolor = 'rgba(255,51,51,.2)',
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        # CVaR Area
        fig.add_annotation(
            x=self.histVaR*100,
            y=0.5,
            xref='x',
            yref='paper',
            text = "CVaR<br>Area",
            ax=-40,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        fig.update_layout(template = 'plotly_white',width=600, height=500, title=title)
        fig.update_shapes(dict(xref='x',yref='paper'))
        fig.show()


class Security(FinancialInstrument):
    def __init__(self, name, identifier, prices=None, returns=None,total_return=None, mu=None, sigma=None, histVaR=None, histCVaR=None, key=None, conf_level=None, time_horizen=None):
        super().__init__(name, identifier, prices, returns,total_return, mu, sigma, histVaR, histCVaR)
        self.conf_level = conf_level
        self.time_horizen = time_horizen
        self.key = key

    def __str__(self):
        """Print Security Attributes"""
        # inherited attributes
        txt = super().__str__()

        try:
            # calcualte the lenght of historical daily data
            from_ = min(self.prices.index).date()
            to_ = max(self.prices.index).date()
            datediff = relativedelta.relativedelta(to_,from_)

            if datediff.years > 1:
                freq = f"{datediff.years:.0f} Years, {datediff.months:.0f} Months"
            else:
                freq = f"{datediff.days} days"
        
            # print message
            # txt = f"{self.name}({self.identifier})\n"
            # txt += "="*100 + "\n"
            txt += f"Date Range:{from_} - {to_} ({freq})\n"
            txt += f"Total Return: {self.total_return*100:.2f}%\n"
            txt += f"Annualized Return: {self.mu*100:.2f}% | Annualized Volatility: {self.sigma*100:.2f}%\n"
            txt += f"Historical VaR ({self.time_horizen} days): {self.histVaR*100:.2f}% | Historical CVaR: {self.histCVaR*100:.2f}%\n"
        except:
            # txt = f"{self.name}({self.identifier})\n"
            txt += "=> Get historical returns or simulate returns"
        return txt
    
    def get_historical_price_data(self):
        """
        Returns daily data for a stock (symbol)
        key: api key
        symbols: VMW,AAPL,GOOG,SPY
        """
        def calc_return(returns):
            """ calculate annualized return from daily returns"""
            return np.prod(1 + returns) ** (252/len(returns)) - 1
        def calc_volatility(returns):
            """calculate annualized volatilit from daily returns"""
            return np.std(returns) * np.sqrt(252)
        def calc_total_return(self):
            """calculate the total return for the period"""
            index_first = self.prices.index[0]
            index_last = self.prices.index[-1]
            return self.prices.loc[index_last,'close'] / self.prices.loc[index_first, 'close'] -1

        # request data from alphavantage
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={self.identifier}&apikey={self.key}&outputsize=full"
        r = requests.get(url)
        d = r.json()

        # extract data to a df
        df = pd.DataFrame(d['Time Series (Daily)']).T
        df.columns = ['open','high','low','close','volume']
        df['symbol'] = self.identifier

        # change data types
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

        # convert datatype to float
        for col in ['open','high','low','close','volume']:
            df[col] = df[col].astype('float')

        # calc daily returns
        df['returns'] = df['close'].pct_change()

        # update the security attributes
        self.prices = df
        self.returns = np.array(df['returns'].dropna())
        self.mu = calc_return(self.returns)
        self.sigma = calc_volatility(self.returns)
        self.total_return = calc_total_return(self)
        self.histVaR = super().calc_HistVaR()
        self.histCVaR = super().calc_HistCVaR()
       



In [4]:
g = Security(
    name ='Gold ETF',
    identifier ='GOLD',
    key = KEY,
    conf_level = CONF_LEVEL, 
    time_horizen = TIME_HORIZEN
)

# get historical data
g.get_historical_price_data()

# # calc Value-at-Risk
# g.calc_HistVaR()
# g.calc_HistCVaR()

# # plots
print(g)
g.plot_prices()
g.plot_histograms()

Gold ETF (GOLD)
Date Range:1999-11-01 - 2024-04-16 (24 Years, 5 Months)
Total Return: -7.47%
Annualized Return: -0.32% | Annualized Volatility: 41.03%
Historical VaR (1 days): -3.83% | Historical CVaR: -5.67%



In [5]:
# Security 
s = Security(
    name ='S%P500',
    identifier ='SPY',
    key = KEY,
    conf_level = CONF_LEVEL, 
    time_horizen = TIME_HORIZEN
)

# get historical data
s.get_historical_price_data()

# plots
print(s)
s.plot_prices()
s.plot_histograms()



S%P500 (SPY)
Date Range:1999-11-01 - 2024-04-16 (24 Years, 5 Months)
Total Return: 271.44%
Annualized Return: 5.52% | Annualized Volatility: 19.54%
Historical VaR (1 days): -1.93% | Historical CVaR: -2.95%



## Monte Carlo Value-at-Risk

In [13]:
class Portfolio(FinancialInstrument):
    def __init__(self, name, identifier, initial_value, securities, weights, n_sims=0, prices=None, returns=None,total_return=None, mu=None, sigma=None, histVaR=None, histCVaR=None, conf_level=None, time_horizen=None):
        super().__init__(name, identifier, prices, returns,total_return, mu, sigma, histVaR, histCVaR)
        self.initial_value = initial_value
        self.securities = securities
        self.weights = weights
        self.n_sims = n_sims
        self.conf_level = conf_level
        self.time_horizen = time_horizen

    def __str__(self):
        txt = super().__str__()
        txt += f"Starting Value: ${self.initial_value:,.0f}\n"
        txt += f"Securities: {[sec.name for sec in self.securities]}\n"
        txt += f"Weights: {[w*100 for w in self.weights]}"
        return txt

p = Portfolio(
    name = 'Equity Portfolio',
    identifier = 'PORT1',
    initial_value = 100000,
    securities = [s,g],
    weights = [0.80,0.20]

)
print(p)


Equity Portfolio (PORT1)
Starting Value: $100,000
Securities: ['S%P500', 'Gold ETF']
Weights: [80.0, 20.0]


In [16]:
class Portfolio:
    def __init__(self, name, securities, weights, initial_value, conf_level,time_horizen, n_sims):
        self.name = name
        self.securities = securities
        self.initial_value = initial_value
        self.weights = weights
        self.conf_level = conf_level
        self.time_horizen = time_horizen
        self.n_sims = n_sims
        self.cov = self.calc_covariance()
        self.hist_total_return = None
        self.hist_return = None
        self.hist_volatility = None 
        self.hist_daily_returns = None
        self.calc_portfolio_values()
        self.HistVaR = self.calc_HistVaR()
        self.HistCVaR = self.calc_HistCVaR()

    def __str__(self):
        txt = f'Portfolio:{self.name}\n'
        txt += "="*100 + '\n'
        txt += f"Historical| Total Return {self.hist_total_return*100:.2f}% | Annualized {self.hist_return*100:.2f}% | Volatility:{self.hist_volatility*100:.2f}%\n"
        txt += f"Historical| Value-at-Risk ({self.time_horizen} days): {self.HistVaR*100:.2f}%\n"
        txt += f"Historical| Conditional Value-at-Risk ({self.time_horizen} days): {self.HistCVaR*100:.2f}%\n"
        return txt

    def calc_covariance(self):
        for i,sec in enumerate(self.securities):
            r = pd.DataFrame(sec.prices['returns'])
            r.columns = [f'returns_{sec.identifier}']
            if i==0:
                df_r = r
            else:
                df_r = df_r.join(r, how='inner',lsuffix=f'_{sec.identifier}')
        # update the cov matrix for the portfolio
        return df_r.cov()

    def calc_portfolio_values(self):
        """ """ 
        # extract historical security returns
        sec_returns = []
        for sec in self.securities:
            sec_returns.append(sec.returns)

        # transpose
        sec_returns = np.array(sec_returns).T

        # calculate the weight portfolio return
        p_returns = sec_returns.dot(self.weights)

        # calculate portfolio values
        p_values = np.cumprod(1 + p_returns) * self.initial_value

        # calculate returns
        p_total_return = (1+p_returns).prod()-1
        p_return  = p_total_return ** (255/len(p_returns)) -1
        p_vol = np.std(p_returns) *  np.sqrt(255)

        # update portfolio attributes
        self.hist_total_return = p_total_return
        self.hist_return = p_return
        self.hist_volatility = p_vol
        self.hist_daily_returns = p_returns

    def calc_HistVaR(self):
        """Historical Value-at-Risk for a confidence interval"""
        VaR = np.percentile(self.hist_daily_returns,100-self.conf_level) * np.sqrt(self.time_horizen)
        return VaR

    def calc_HistCVaR(self):
        """
        Historical Conditional Value-at-Risk for a confidence interval
        Find the average of the worst returns below the VaR
        """
        below_var = self.hist_daily_returns <= self.HistVaR
        CVaR = self.hist_daily_returns[below_var].mean() * np.sqrt(self.time_horizen)
   
        return CVaR

    def plot_histograms(self):
        """"Plot the histogram of daily returns with VaR"""

        title = f"<b>{self.name} Historical VaR ({self.time_horizen} days) </b><br>VaR:{self.HistVaR*100:.2f}% | CVaR: {self.HistCVaR*100:.2f}%"
        returns = self.hist_daily_returns *100
        min_r = min(self.hist_daily_returns)*100
        max_r = max(self.hist_daily_returns)*100

        # histogram
        fig = go.Figure(
            go.Histogram(
                x = returns,
                marker_color = 'rgba(83,128,141,0.8)'
            )
        )
        # CVaR Area
        fig.add_shape(
            type='rect',
            x0=min_r,
            x1= self.HistVaR*100,
            y0=0,
            y1=1,
            fillcolor='red',
            opacity=0.1
        )
        # Non-CVaR Area
        fig.add_shape(
            type='rect',
            x1=max_r,
            x0= self.HistVaR*100,
            y0=0,
            y1=1,
            fillcolor='skyblue',
            opacity=0.1
        )
        # VaR threshold
        fig.add_shape(
            type='line',
            x1=self.HistVaR *100,
            x0= self.HistVaR*100,
            y0=0,
            y1=1,
            line=dict(color='rgba(255,51,51,.5)'),
        )

        # VaR Threshold
        fig.add_annotation(
            x=self.HistVaR*100,
            y=0.8,
            xref='x',
            yref='paper',
            text = "VaR<br>Threshold",
            ax=100,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            bordercolor = 'rgba(255,51,51,.2)',
            bgcolor = 'rgba(255,51,51,.2)',
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        # CVaR Area
        fig.add_annotation(
            x=self.HistVaR*100,
            y=0.5,
            xref='x',
            yref='paper',
            text = "CVaR<br>Area",
            ax=-40,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        fig.update_layout(template = 'plotly_white',width=600, height=500, title=title)
        fig.update_shapes(dict(xref='x',yref='paper'))
        fig.show()
       
# Portfolio
portfolio = Portfolio(
    name = 'Equity Portfolio', 
    securities= [s,g],
    weights = [0.80, 0.20],
    initial_value = 100000,
    conf_level = CONF_LEVEL,
    time_horizen = TIME_HORIZEN,
    n_sims = 10
)


print(portfolio)
portfolio.plot_histograms()



Portfolio:Equity Portfolio
Historical| Total Return 304.57% | Annualized 4.73% | Volatility:18.89%
Historical| Value-at-Risk (1 days): -1.78%
Historical| Conditional Value-at-Risk (1 days): -2.78%



## Variance/Covariance Value-at-Risk

## Conditional Value-at-Risk